In [ ]:
import pandas as pd
from pymongo import MongoClient
import requests
import json

In [2]:
# create a MongoDB client instance
client = MongoClient('mongodb://localhost:27017')

# get a reference to the database
db = client.mydatabase

# get a reference to the collection
collection = db.closing_prices_latest

# define your Alpha Vantage API key and endpoint URL
API_KEY = "HMU3C6GSU9RRXY5Z"
url = "https://www.alphavantage.co/query"

# define the symbol and data type to retrieve
symbol = ["AAPL","PAYTM.BSE"]
data_type = "TIME_SERIES_DAILY_ADJUSTED"

20

In [ ]:
# append new data to the collection
def updateDocument(symbol):
    for i in range(len(symbol)):
            # define the API parameters
        params = {
            "function": data_type,
            "symbol": symbol[i],
            "apikey": API_KEY
        }
        # make a request to the Alpha Vantage API
        response = requests.get(url, params=params)

        # convert the response data to a Python dictionary
        data = json.loads(response.text)

        # get the latest timestamp in the collection, if any
        latest_timestamp = collection.find_one(sort=[("timestamp", -1)])["timestamp"] if collection.count_documents({}) > 0 else None

        for date, values in data["Time Series (Daily)"].items():
            timestamp = pd.to_datetime(date)
            if latest_timestamp is None or timestamp > latest_timestamp:
                document = {
                    "symbol": symbol[1],
                    "timestamp": timestamp,
                    "close": float(values["4. close"])
                }
                collection.update_one({"timestamp": timestamp}, {"$set": document}, upsert=True)

In [ ]:
# query = "{ qty: { $gt: 4 } }"  # define your query here if necessary

results = collection.find( { "symbol": symbol[1] ,"close" :{ "$gt": 150} } )

df = pd.DataFrame(list(results))

print(df)